## Pydantic Output Parser

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)
load_dotenv('./../.env')

True

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.5,
    max_tokens=300
)

### Import Pydantic Libraries

In [3]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import chain
from langchain_core.messages import AIMessage
import re

### Create Joke class as extension of BaseModel

In [4]:
class Joke(BaseModel):
    """Joke Model to Entertain the User"""
    
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="Punchline of the joke")
    rating: Optional[int] = Field(description="Rating of the joke, from 1 to 10", examples=(5, 7))


### Create Parser using the PydanticOutputParser

In [5]:
parser = PydanticOutputParser(pydantic_object=Joke)

instruction = parser.get_format_instructions()

print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke Model to Entertain the User", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "Punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "description": "Rating of the joke, from 1 to 10", "examples": [5, 7], "title": "Rating"}}, "required": ["setup", "punchline", "rating"]}
```


### Provide LLM Input Prompt & Parse Response

In [ ]:
prompt = PromptTemplate(
    template=
        """ 
        Answer the user query with a joke. Following is your formatting insturction.
        {format_instruction}
        
        Query: {query}
        Answer:
        """,
    input_variables=['query'],
    partial_variables={'format_instruction': instruction}
)

@chain
def remove_think_block_ai_message(AIMessage) -> str:
    return re.sub(r"<think>.*?</think>", "", AIMessage.content, flags=re.DOTALL | re.IGNORECASE)

chain = prompt | llm | remove_think_block_ai_message | parser

response = chain.invoke({'query': "Tell a joke about cats"})

print(response)

### Parse LLM response using WithStructuredOutput Method

In [7]:
structured_llm = llm.with_structured_output(Joke)

output = structured_llm.invoke("Tell a joke about cats")

print(output)

setup="Why don't cats play hide and seek with the furniture?" punchline='Because the furniture always wins.' rating=5


### Parse LLM Response using JsonOutputParser to JSON Object

In [8]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template=
        """ 
        Answer the user query with a joke. Following is your formatting insturction.
        {format_instruction}
        
        Query: {query}
        Answer:
        """,
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = prompt | llm | remove_think_block_ai_message | parser

response = chain.invoke({'query': "Tell a joke about Dogs"})

print(response)


{'setup': "Why don't dogs make good secret agents?", 'punchline': 'Because they always bark!', 'rating': 7}


### Use of CSV Output Parser

In [27]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.runnables import chain

parser = CommaSeparatedListOutputParser()

format_instruction = parser.get_format_instructions()

prompt = PromptTemplate(
    template=
        """ 
        Answer the user query with a list of values. Following is your formatting insturction.
        {format_instruction}
        
        Query: {query}
        Answer:
        """,
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = prompt | llm | remove_think_block_ai_message | parser

response = chain.invoke({"query": "Who were the world's top 5 richest persons in 2022 ?"})

print(response)

['Elon Musk', 'Bernard Arnault & Family', 'Jeff Bezos', 'Amancio Ortega', 'Bill Gates']


### Use of DateTime Output Parser

In [45]:
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser(format="%d-%m-%Y")

format_instruction = parser.get_format_instructions()

prompt = PromptTemplate(
    template=
        """ 
        Answer the user query with a correct date. Following is your formatting insturction.
        {format_instruction}
        
        Query: {query}
        Answer:
        """,
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = prompt | llm | remove_think_block_ai_message | parser

response = chain.invoke({"query": "When did Singapore get Independence ?"})

print(response)

1965-08-09 00:00:00
